In [1]:
!pip install pandas numpy torch matplotlib hdbscan umap-learn scikit-learn sentence-transformers
!pip install bertopic==0.16.2
!pip install demoji
!pip install advertools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.3/252.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/

In [ ]:
!pip install evaluate
!pip install bert-score==0.3.13
!pip install rouge-score==0.1.2

In [2]:
!pip install -qq accelerate
!pip install -qq datasets
!pip install -qq trl
!pip install -qq transformers
!pip install -qq bitsandbytes

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import demoji
from statistics import mean
from torch import cuda
from transformers import (AutoModelForCausalLM,
                          AutoModelForSeq2SeqLM,
                          AutoTokenizer,
                          GenerationConfig,
                          BitsAndBytesConfig,
                          pipeline,
                          logging)
from datasets import Dataset
from bertopic import BERTopic
from statistics import mean
import torch
import matplotlib.pyplot as plt
import hdbscan
import os
import io
import re
import string
import nltk
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize
from umap import UMAP
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import seaborn as sns
import advertools as adv
from bertopic.representation import TextGeneration
from bertopic.vectorizers import ClassTfidfTransformer
from evaluate import load
from itertools import product
from statistics import mean
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/scrapy/utils/_compression.py:18: ScrapyDeprecationWarning: You have brotlipy installed, and Scrapy will use it, but Scrapy support for brotlipy is deprecated and will stop working in a future version of Scrapy. brotlipy itself is deprecated, it has been superseded by brotlicffi. Please, uninstall brotlipy and install brotli or brotlicffi instead. brotlipy has the same import name as brotli, so keeping both installed is strongly discouraged.
  warn(


In [4]:
def remove_emojis(text):
    return demoji.replace(text, '')

def preprocess_text(text):
    # Remove retweets
    text = re.sub(r'^RT[\s]+', '', text, flags=re.IGNORECASE)
    # Remove usernames @
    text = re.sub(r'@[^\s]+', '', text)
    # Remove URLs
    text = re.sub(r'https\S+', '', text)
    url_words = ['url', 'URL', 'html', 'HTML', 'http', 'HTTP']
    for u in url_words:
        text = re.sub(u, '', text)
    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)
    text = remove_emojis(text)
    # Remove extra spaces that may have been introduced
    text = re.sub(r'\s+', ' ', text).strip()
    # Return preprocessed text
    return text

def load_dataset(input_file, stance, topic) -> pd.DataFrame:
    '''
    Each time we only deal with a single topic and stance.
    '''
    input_corpus = pd.read_csv(input_file)
    input_corpus = input_corpus[input_corpus["topic"] == topic]

    if stance == 1:
        input_corpus = input_corpus[input_corpus["stance"] == 1]
    else:
        input_corpus = input_corpus[input_corpus["stance"] == -1]

    input_corpus.reset_index(drop=True, inplace=True)

    # Debugging: Check if DataFrame is empty after filtering
    if input_corpus.empty:
        print("No data available for the specified topic and stance.")
        return pd.DataFrame()  # Return an empty DataFrame

    # Preprocess the arguments and store them in a new column
    preprocessed_data = [preprocess_text(arg) for arg in input_corpus['argument']]

    # Debugging: Check preprocessed data
    if not preprocessed_data:
        print("Preprocessed data is empty. Check the preprocessing function.")

    # Expand the tuples into separate columns
    input_corpus['preprocessed_arguments']= preprocessed_data

    # Debug output
    print("Loaded dataset:")
    print(input_corpus.head())
    print(f"Total records: {len(input_corpus)}")

    return input_corpus

# T5 baseine

In [5]:
model_id = 'IMISLab/GreekWiki-umt5-base'

main_prompt = """
summarize:
[DOCUMENTS]
"""
prompt = main_prompt

tokenizer=AutoTokenizer.from_pretrained(model_id)
t5_base=model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
# Create your representation model
generator = pipeline(task='text2text-generation',
                     model=t5_base,
                     tokenizer=tokenizer,
                     max_new_tokens=100,
                     num_beams=3,
                     early_stopping=True,
                     length_penalty= 2.0,
                     device="cuda")
greek_t5_base = TextGeneration(generator,prompt=prompt)


tokenizer_config.json:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.55M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [6]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HUGGINGFACE_TOKEN")
!huggingface-cli login --token $secret_hf

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Meltemi_base baseline

In [7]:
summarization_prompt = """
Παρακάτω θα δεις μερικά επιχειρήματα υπέρ ή κατά για ένα συγκεκριμένο θέμα:
[DOCUMENTS]
Με βάση τα παραπάνω, γράψε μια σύντομη πρόταση που να συνοψίζει αυτά τα επιχειρήματα σε ένα keypoint, ακολουθώντας το μοτίβο:
θέμα: <keypoint>"""

In [8]:
model_id = "ilsp/Meltemi-7B-v1.5"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)
# Llama 2 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
base_model.eval()

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"

tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE
messages = [
    {"role": "system", "content": "Είσαι το Μελτέμι, ένα γλωσσικό μοντέλο για την ελληνική γλώσσα. Είσαι ιδιαίτερα βοηθητικό προς την χρήστρια ή τον χρήστη και δίνεις σύντομες αλλά επαρκώς περιεκτικές απαντήσεις. Απάντα με προσοχή, ευγένεια, αμεροληψία, ειλικρίνεια και σεβασμό προς την χρήστρια ή τον χρήστη."},
    {"role": "user", "content": summarization_prompt},
]
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

# Our text generator
generator = pipeline(
    model=base_model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=100,
    num_beams=3,
    early_stopping=True,
    length_penalty= 2.0)
# Text generation with Zephyr
meltemi_base = TextGeneration(generator, prompt=prompt)

tokenizer_config.json:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Meltemi_Instruct baseline

In [9]:
model_id = "ilsp/Meltemi-7B-Instruct-v1.5"
# Llama 2 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Llama 2 Model
instruct_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
instruct_model.eval()

messages = [
    {"role": "system", "content": "Είσαι το Μελτέμι, ένα γλωσσικό μοντέλο για την ελληνική γλώσσα. Είσαι ιδιαίτερα βοηθητικό προς την χρήστρια ή τον χρήστη και δίνεις σύντομες αλλά επαρκώς περιεκτικές απαντήσεις. Απάντα με προσοχή, ευγένεια, αμεροληψία, ειλικρίνεια και σεβασμό προς την χρήστρια ή τον χρήστη."},
    {"role": "user", "content": summarization_prompt},
]


prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
# Our text generator
generator = pipeline(
    model=instruct_model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=100,
    num_beams=3,
    early_stopping=True,
    length_penalty= 2.0
)
meltemi_instruct = TextGeneration(generator, prompt=prompt)

tokenizer_config.json:   0%|          | 0.00/3.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Few shot Meltemi instruct baseline

In [10]:
from datasets import Dataset

def create_huggingface_dataset(data_set, argument_column='arg'):
    # Step 1: Filter to keep only rows with label '1'
    filtered_data = data_set[data_set['label'] == 1]

    # Step 2: Group by 'key_point', 'topic', and 'stance' and count arguments
    # Keep only the first 4 arguments in each group
    grouped_data = filtered_data.groupby(['key_point', 'topic', 'stance'])[argument_column] \
        .apply(lambda x: '\n'.join([f"- {arg}" for arg in x[:4]])).reset_index()

    # Step 3: Count the number of arguments for each 'key_point' (if needed)
    argument_counts = filtered_data.groupby(['key_point', 'topic', 'stance']).size().reset_index(name='count')

    # Step 4: Merge the argument counts with grouped data (optional, can be removed if not needed)
    grouped_data = grouped_data.merge(argument_counts, on=['key_point', 'topic', 'stance'])

    # Step 5: Create a Hugging Face Dataset including 'key_point', 'topic', 'stance', and the formatted arguments
    hf_dataset = Dataset.from_pandas(grouped_data[['key_point', 'topic', 'stance', argument_column]])

    return hf_dataset

# Example Usage:
# Load your train and dev DataFrames
train_df = pd.read_csv('/kaggle/input/meltemi-data/train_dev_test_dataset/final_data/train_gr.csv')
dev_df = pd.read_csv('/kaggle/input/meltemi-data/train_dev_test_dataset/final_data/dev_human_gr.csv')

# Create Hugging Face datasets
train_hf_dataset = create_huggingface_dataset(train_df)
dev_hf_dataset = create_huggingface_dataset(dev_df)


def get_filtered_rows(hf_dataset):
    # Convert the Hugging Face Dataset to a pandas DataFrame for easier manipulation
    df = hf_dataset.to_pandas()

    # Step 1: Find topics that contain at least one 'for' (stance = 1) and one 'against' (stance = -1)
    qualifying_topics = []
    for topic in df['topic'].unique():
        topic_df = df[df['topic'] == topic]
        
        # Check if the topic has at least one positive and one negative stance
        has_positive = not topic_df[topic_df['stance'] == 1].empty
        has_negative = not topic_df[topic_df['stance'] == -1].empty
        
        if has_positive and has_negative:
            qualifying_topics.append(topic)
        
        # Stop once we have 2 qualifying topics
        if len(qualifying_topics) == 2:
            break

    # Step 2: Filter the DataFrame to include only the selected topics
    filtered_df = df[df['topic'].isin(qualifying_topics)]

    # Step 3: For each selected topic, get the first positive and first negative stance
    result_df = pd.DataFrame()  # Initialize an empty DataFrame to store results
    for topic in qualifying_topics:
        topic_df = filtered_df[filtered_df['topic'] == topic]
        
        # Get the first row with stance -1 and stance 1
        first_negative = topic_df[topic_df['stance'] == -1].head(1)
        first_positive = topic_df[topic_df['stance'] == 1].head(1)
        
        # Concatenate the results to the result DataFrame
        result_df = pd.concat([result_df, first_negative, first_positive])

    # Step 4: Convert the result back to a Hugging Face Dataset
    filtered_hf_dataset = Dataset.from_pandas(result_df.reset_index(drop=True))

    return filtered_hf_dataset

# Apply the function to get the filtered dataset
filtered_train_hf_dataset = get_filtered_rows(train_hf_dataset)


In [11]:
# Define the system message
system_message = {
    "role": "system",
    "content": "Είσαι το Μελτέμι, ένα γλωσσικό μοντέλο για την ελληνική γλώσσα. "
               "Είσαι ιδιαίτερα βοηθητικό προς την χρήστρια ή τον χρήστη και δίνεις σύντομες αλλά επαρκώς περιεκτικές απαντήσεις. "
               "Απάντα με προσοχή, ευγένεια, αμεροληψία, ειλικρίνεια και σεβασμό προς την χρήστρια ή τον χρήστη."
}

# Example template to structure demonstration examples
example_template = """Παρακάτω θα δεις μερικά επιχειρήματα υπέρ ή κατά για ένα συγκεκριμένο θέμα:
[ARGUMENTS]

Με βάση τα παραπάνω, γράψε μια σύντομη πρόταση που να συνοψίζει αυτά τα επιχειρήματα σε ένα keypoint, ακολουθώντας το μοτίβο:
θέμα: <keypoint>"""

# Function to load demonstrations from the dataset as message entries
def load_demonstrations_as_messages(hf_dataset):
    # Convert Hugging Face dataset to pandas DataFrame for easy access
    df = hf_dataset.to_pandas()

    # Create list of messages, starting with the system message
    messages = [system_message]

    # Build demonstration examples, including both user prompts and assistant responses
    for _, row in df.iterrows():
        # Format arguments as a bullet-point list
        arguments_text = '\n'.join(f"{arg}" for arg in row['arg'].split('\n'))
        #topic_text=row['topic']
        # Construct the user and assistant messages for each demonstration
        example_user_text = example_template.replace("[ARGUMENTS]", arguments_text) #.replace("TOPIC",topic_text)
        example_assistant_text = "θέμα: " +row['key_point']
        
        # Append the user and assistant demonstration messages
        messages.append({"role": "user", "content": example_user_text})
        messages.append({"role": "assistant", "content": example_assistant_text})

    # Main prompt to be added as the final user message
    main_prompt = """Παρακάτω θα δεις μερικά επιχειρήματα υπέρ ή κατά για ένα συγκεκριμένο θέμα:
[DOCUMENTS]

Με βάση τα παραπάνω, γράψε μια σύντομη πρόταση που να συνοψίζει αυτά τα επιχειρήματα σε ένα keypoint, ακολουθώντας το μοτίβο:
θέμα: <keypoint>"""
    messages.append({"role": "user", "content": main_prompt})

    return messages

# Apply the chat template
messages = load_demonstrations_as_messages(filtered_train_hf_dataset)
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
print(prompt)

<|system|>
Είσαι το Μελτέμι, ένα γλωσσικό μοντέλο για την ελληνική γλώσσα. Είσαι ιδιαίτερα βοηθητικό προς την χρήστρια ή τον χρήστη και δίνεις σύντομες αλλά επαρκώς περιεκτικές απαντήσεις. Απάντα με προσοχή, ευγένεια, αμεροληψία, ειλικρίνεια και σεβασμό προς την χρήστρια ή τον χρήστη.</s>
<|user|>
Παρακάτω θα δεις μερικά επιχειρήματα υπέρ ή κατά για ένα συγκεκριμένο θέμα:
- Το κλείσιμο του θα οδηγήσει μόνο σε περισσότερους τρομοκράτες που θα τρέχουν ελεύθεροι και έτσι θα θέσουν σε κίνδυνο την ασφάλειά μας.
- Η κράτηση τρομοκρατών στο Γκουαντάναμο εξασφαλίζει ότι δεν μπορούν να σχεδιάσουν μελλοντικές επιθέσεις εναντίον της Αμερικής.
- Η κράτηση τρομοκρατών στο Γκουαντάναμο τους κρατάει μακριά από το πεδίο της μάχης.
- Για λόγους ασφαλείας πρέπει να κρατήσουμε το στρατόπεδο ανοιχτό.

Με βάση τα παραπάνω, γράψε μια σύντομη πρόταση που να συνοψίζει αυτά τα επιχειρήματα σε ένα keypoint, ακολουθώντας το μοτίβο:
θέμα: <keypoint></s>
<|assistant|>
θέμα: Το κλείσιμο του στρατοπέδου κράτησης του

In [12]:
# Our text generator
generator = pipeline(
    model=instruct_model,
    tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=100,
    num_beams=3,
    early_stopping=True,
    length_penalty= 2.0
)
meltemi_instruct_few = TextGeneration(generator, prompt=prompt)

# Topic modelling with BERTopic

In [ ]:
# Topic Modeling Script for Jupyter Notebook
# --------------------------------------------------------------
n_neighbors = 10  # Set your desired value
n_components = 2  # Set your desired value
min_samples_fraction = 1.0  # Set your desired value (as a fraction)
selected_arguments_path = '/kaggle/input/meltemi-data/'
output_topic_data = './output_topic_data/'  # Output directory for topic data
output_arguments_data = './output_arguments_data/'  # Output directory for arguments data
stopwords = list(adv.stopwords['greek'])

# Set environment variable to avoid parallelism issues with tokenizers
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Set device for torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def topic_modeling(df, output_prefix, topic, stance):
    cluster_size = int(len(df) / 50)
    if cluster_size < 3:
        cluster_size = 3

    print(f"Cluster size: {cluster_size}")

    min_samples = int(min_samples_fraction * cluster_size)
    if min_samples < 2:
        min_samples = 2

    print(f"Min samples: {min_samples}")

    text_list = df['preprocessed_arguments'].tolist()
    embedding_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2", trust_remote_code=True)
    embeddings = embedding_model.encode(text_list)
    umap_model = UMAP(random_state=42, n_neighbors=n_neighbors, n_components=n_components, min_dist=0.00, metric='cosine')
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=cluster_size,
                                    metric='euclidean',
                                    cluster_selection_method='leaf',
                                    min_samples=min_samples,
                                    prediction_data=True)
    vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords, lowercase=True)
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
    representation_model = {"Greek_t5_0shot": greek_t5_base,"Meltemi_base_0shot": meltemi_base,"Meltemi_Instruct_0shot": meltemi_instruct,"Meltemi_Instruct_4shot": meltemi_instruct_few}
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        representation_model=representation_model,
        low_memory=True,
        nr_topics=10,
        verbose=True,
        calculate_probabilities=True,
        language="multilingual"
    )

    topics, probabilities = topic_model.fit_transform(documents=text_list, embeddings=embeddings)

    # Ensure output directory exists
    os.makedirs(output_topic_data, exist_ok=True)

    # Save topic information to CSV
    topic_info = topic_model.get_topic_info()

    # Add 'topic' and 'stance' columns to the topic information
    topic_info['topic'] = topic
    topic_info['stance'] = stance

    # Save the updated topic information to CSV
    output_file = os.path.join(output_topic_data, f"{output_prefix}_topic_info.csv")
    topic_info.to_csv(output_file, index=False)
    print(f"Topic information saved to {output_file}")

# Load the main CSV file that contains topics and stances
arguments_file_path = os.path.join(selected_arguments_path, 'arguments_human_translated_test.csv')
arguments_df = pd.read_csv(arguments_file_path)

# Get unique combinations of topic and stance
unique_combinations = arguments_df[['topic', 'stance']].drop_duplicates()

# Iterate through each unique combination of topic and stance
for _, row in unique_combinations.iterrows():
    topic = row['topic']
    stance = row['stance']

    # Load dataset based on the topic and stance
    filtered_df = load_dataset(arguments_file_path, stance, topic)

    # Perform topic modeling and save results
    topic_modeling(filtered_df, output_prefix=f"{topic}_{stance}", topic=topic, stance=stance)

Loaded dataset:
    arg_id                                           argument  \
0  arg_0_0  Οι εμβολιασμοί ρουτίνας δεν είναι απαραίτητοι ...   
1  arg_0_1  Οι παιδικοί εμβολιασμοί ρουτίνας δεν θα πρέπει...   
2  arg_0_2  Οι εμβολιασμοί ρουτίνας των παιδιών δεν χρειάζ...   
3  arg_0_3  Ένα εμβόλιο, το οποίο δεν έχει ελεγχθεί επαρκώ...   
4  arg_0_4  Εφόσον τα εμβόλια φέρουν επιπτώσεις, δεν μπορο...   

                                               topic  stance  \
0  Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...      -1   
1  Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...      -1   
2  Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...      -1   
3  Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...      -1   
4  Οι παιδικοί εμβολιασμοί ρουτίνας θα πρέπει να ...      -1   

                              preprocessed_arguments  
0  Οι εμβολιασμοί ρουτίνας δεν είναι απαραίτητοι ...  
1  Οι παιδικοί εμβολιασμοί ρουτίνας δεν θα πρέπει...  
2  Οι εμβολιασμοί ρουτίνας των παιδιών δεν 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

2024-11-21 10:48:58,531 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-21 10:48:58,618 - BERTopic - Dimensionality - Completed ✓
2024-11-21 10:48:58,619 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-21 10:48:58,627 - BERTopic - Cluster - Completed ✓
2024-11-21 10:48:58,627 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 8/8 [02:06<00:00, 15.75s/it]
2024-11-21 11:01:51,931 - BERTopic - Representation - Completed ✓
2024-11-21 11:01:51,932 - BERTopic - Topic reduction - Reducing number of topics
2024-11-21 11:01:51,933 - BERTopic - Topic reduction - Reduced number of topics from 8 to 8


Topic information saved to ./output_topic_data/Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς_-1_topic_info.csv
Loaded dataset:
     arg_id                                           argument  \
0  arg_2_66  Εκτός από την ποικιλομορφία των ανθρώπων, οι Η...   
1  arg_2_67  Η Αμερική είναι μια χώρα με πολλές ευκαιρίες γ...   
2  arg_2_68  Η Αμερική είναι ένα ελεύθερο έθνος όπου δουλεύ...   
3  arg_2_69  Παρά τους υψηλούς φόρους, η ζωή στις Ηνωμένες ...   
4  arg_2_70  Τέλεια χώρα, τέλειοι άνθρωποι. Δεν θα την άλλα...   

                                      topic  stance  \
0  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς       1   
1  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς       1   
2  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς       1   
3  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς       1   
4  Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς       1   

                              preprocessed_arguments  
0  Εκτός από την ποικιλομορφία των ανθρώπων, οι Η...  
1  Η Αμερική είναι μια χώρα με πολλές ευ

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

2024-11-21 11:01:55,752 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-21 11:01:55,953 - BERTopic - Dimensionality - Completed ✓
2024-11-21 11:01:55,954 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-21 11:01:55,966 - BERTopic - Cluster - Completed ✓
2024-11-21 11:01:55,967 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 18/18 [04:44<00:00, 15.82s/it]
2024-11-21 11:29:45,501 - BERTopic - Representation - Completed ✓
2024-11-21 11:29:45,502 - BERTopic - Topic reduction - Reducing number of topics
100%|██████████| 10/10 [02:44<00:00, 16.48s/it]
2024-11-21 11:45:46,607 - BERTopic - Topic reduction - Reduced number of topics from 18 to 10


Topic information saved to ./output_topic_data/Οι ΗΠΑ είναι μια καλή χώρα να ζει κανείς_1_topic_info.csv


In [14]:
# Define the directory where your CSV files are located
csv_directory = './output_topic_data/'  # Replace with the path to your CSV files
merged_output_file = './final_comparison_test.csv'  # Path for the merged output file

# Create an empty list to hold DataFrames
dataframes = []

# Iterate over all files in the directory
for file in os.listdir(csv_directory):
    if file.endswith('.csv'):
        file_path = os.path.join(csv_directory, file)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dataframes, ignore_index=True)
merged_df.to_csv(merged_output_file, index=False)

#   Extracting the first complete sentence out of each generated text of each cluster for each model (GreekT5, Meltemi_base,Meltemi_Instruct). 
#   Compute Rouge and BertScores

In [ ]:
def preprocess_t5(text):
    # If the input is a list, convert it to a string using only the first non-empty element
    if isinstance(text, list):
        # Join non-empty elements into a single string and strip whitespace
        non_empty_elements = [item.strip() for item in text if isinstance(item, str) and item.strip()]
        if non_empty_elements:
            return non_empty_elements[0]  # Return the first non-empty element as a string

    elif isinstance(text, str):
        # Handle the case where the text contains `\n` as a literal
        text = text.replace("\\n", "\n")

        # Split the text at the first occurrence of either a period or an actual newline
        first_part = re.split(r"[.\n]", text, 1)[0]
        text = first_part.strip() + '.' if first_part else ''

    # Final step: Check if the text starts with '['
    if isinstance(text, str) and text.startswith('['):
        # Attempt to extract the string part between single quotes
        match = re.search(r"'(.*?)'", text)
        if match:
            text = match.group(1)  # Return the matched group without quotes
        else:
            # If no single quotes found, simply return the cleaned text as a string
            text = text.strip("[]")  # Remove brackets if present

    # Remove leading and trailing single quotes, if they exist
    text = text.strip("'").strip()  # Remove any leading/trailing single quotes

    # Remove leading hyphen character if it exists
    if text.startswith('-'):
        text = text[1:].strip()  # Remove the leading hyphen and strip whitespace

    return text  # Return the cleaned text

# Apply the preprocessing function to each of the specified columns
for column in ['Greek_t5_0shot']:
    merged_df[column] = merged_df[column].apply(preprocess_t5)

In [ ]:
def preprocess_meltemi_base1(text):
    # Final step: Check if the text starts with '['
    if isinstance(text, str) and text.startswith('['):
        # Remove brackets if present
        text = text.strip("[]")

    # Remove leading and trailing single quotes, if they exist
    text = text.strip("'\"")

    # Remove leading newline characters
    text = text.lstrip("\n")

    # Remove specified prefixes
    prefixes = ["summarize: ", "assist: ", "assistant: "]
    for prefix in prefixes:
        if text.lstrip().startswith(prefix):
            text = text.lstrip()[len(prefix):].strip()  # Remove the prefix and strip whitespace
            break  # Stop after the first match

    return text  # Return the cleaned text





# Apply the preprocessing function to each of the specified columns
for column in ['Meltemi_base_0shot']:
    merged_df[column] = merged_df[column].apply(preprocess_meltemi_base1)

In [ ]:
def preprocess_meltemi_base2(text):
    # If the input is a list, convert it to a string using only the first non-empty element
    if isinstance(text, list):
        # Join non-empty elements into a single string and strip whitespace
        non_empty_elements = [item.strip() for item in text if isinstance(item, str) and item.strip()]
        if non_empty_elements:
            # Return the first non-empty element as a string
            return non_empty_elements[0]

    # If the input is a string, process it
    if isinstance(text, str):
        # Strip leading newline characters if present
        text = text.lstrip('\n')  # Remove any leading newline characters

        # Handle the case where the text contains `\n` as a literal
        text = text.replace("\\n", "\n").strip()  # Replace literal "\n" with actual newline and strip whitespace

        # Search for the specific prefix and extract the following text
        prefixes = ["Παρακάτω θα δεις μερικά επιχειρήματα υπέρ ή κατά για ένα συγκεκριμένο θέμα:","Γράψε μια σύντομη πρόταση ως περίληψη για το παρακάτω κείμενο:", "Τα παρακάτω επιχειρήματα συνοψίζουν ένα συγκεκριμένο θέμα:", "Το παρακάτω επιχείρημα συνοψίζει ένα συγκεκριμένο θέμα:"]
        for prefix in prefixes:
          if prefix in text:
            text = text.split(prefix, 1)[1].strip()

            # Keep only the first sentence after the prefix
            text = text.split('.')[0].strip()  # Split on '.' and keep the first part

        # Remove leading hyphen character if it exists
        if text.startswith('-'):
            text = text[1:].strip()  # Remove the leading hyphen and strip whitespace

        # Remove leading and trailing single quotes, if they exist
        #text = text.strip("'").strip()  # Remove any leading/trailing single quotes

        # Remove content after a space-hyphen pattern
        text = re.split(r'-', text)[0].strip()  # Split on '-' and keep only the content before it

    return text  # Return the cleaned text

# Apply the preprocessing function to each of the specified columns
for column in ['Meltemi_base_0shot']:
    merged_df[column] = merged_df[column].apply(preprocess_meltemi_base2)

In [ ]:
def preprocess_meltemi_base3(text):
    # If the input is a list, convert it to a string using only the first non-empty element
    if isinstance(text, list):
        # Join non-empty elements into a single string and strip whitespace
        non_empty_elements = [item.strip() for item in text if isinstance(item, str) and item.strip()]
        if non_empty_elements:
            # Return the first non-empty element as a string
            return non_empty_elements[0]

    # If the input is a string, process it
    if isinstance(text, str):
        # Strip leading newline characters if present
        text = text.lstrip('\n')  # Remove any leading newline characters

        # Handle the case where the text contains `\n` as a literal
        text = text.replace("\\n", "\n").strip()  # Replace literal "\n" with actual newline and strip whitespace

        # Remove leading hyphen character if it exists
        if text.startswith('-'):
            text = text[1:].strip()  # Remove the leading hyphen and strip whitespace

        # Remove leading and trailing single quotes, if they exist
        #text = text.strip("'").strip()  # Remove any leading/trailing single quotes

        # Remove content after a space-hyphen pattern
        text = re.split(r'-', text)[0].strip()  # Split on '-' and keep only the content before it

        # Keep only the content until the first full stop
        text = text.split('.')[0].strip()  # Split on '.' and keep the first part

    return text  # Return the cleaned text



# Apply the preprocessing function to each of the specified columns
for column in ['Meltemi_base_0shot']:
    merged_df[column] = merged_df[column].apply(preprocess_meltemi_base3)


In [ ]:
def preprocess_meltemi_instruct1(text):
    # Step 1: Ignore everything before the first colon
    if ':' in text:
        text = text.split(':', 1)[1]  # Keep content after the first colon

    # Step 2: Drop leading newline characters
    text = text.lstrip('\n')  # Removes any leading newlines

    # Step 3: Remove any leading numbers and spaces (e.g., '\n\n1. ')
    text = re.sub(r'^\s*\\n\\n\d+\.\s*', '', text)

    # Step 4: Replace any remaining newline characters with a space and strip leading/trailing spaces
    text = text.replace('\n', ' ').strip()

    # Step 5: If the text ends with a full stop, keep content only until the full stop
    if text.endswith('.'):
        text = text.split('.')[0] + '.'  # Keep content up to and including the first full stop

    return text

def preprocess_dataframe(df, column_name):
    # Apply the preprocess_text function to the specified column
    df[column_name] = df[column_name].apply(preprocess_meltemi_instruct1)
    return df

# Example usage
processed_df = preprocess_dataframe(merged_df, 'Meltemi_Instruct_0shot')

In [ ]:
def preprocess_meltemi_instruct2(text):
    # If the input is a list, convert it to a string using only the first non-empty element
    if isinstance(text, list):
        # Join non-empty elements into a single string and strip whitespace
        non_empty_elements = [item.strip() for item in text if isinstance(item, str) and item.strip()]
        if non_empty_elements:
            return non_empty_elements[0]  # Return the first non-empty element as a string

    elif isinstance(text, str):
        # Handle the case where the text contains `\n` as a literal
        text = text.replace("\\n", "\n")

        # Remove the pattern: two newlines, followed by "1. "
        text = re.sub(r'^\n\n\.\s*', '', text)

        # Split the text at the first occurrence of either a period or an actual newline
        first_part = re.split(r"[.\n]", text, 1)[0]
        text = first_part.strip() + '.' if first_part else ''

    # Final step: Check if the text starts with '['
    if isinstance(text, str) and text.startswith('['):
        # Attempt to extract the string part between single quotes
        match = re.search(r"'(.*?)'", text)
        if match:
            text = match.group(1)  # Return the matched group without quotes
        else:
            # If no single quotes found, simply return the cleaned text as a string
            text = text.strip("[]")  # Remove brackets if present

    # Remove leading and trailing single quotes, if they exist
    text = text.strip("'").strip()  # Remove any leading/trailing single quotes

    # Remove leading hyphen character if it exists
    if text.startswith('-'):
        text = text[1:].strip()  # Remove the leading hyphen and strip whitespace

    return text  # Return the cleaned text


# Apply the preprocessing function to each of the specified columns
for column in ['Meltemi_Instruct_0shot']:
    processed_df[column] = processed_df[column].apply(preprocess_meltemi_instruct2)

In [ ]:
def preprocess_text(text):
    # Step 1: Remove square brackets if present
    text = text.strip('[]')

    # Step 2: Split the string by comma and remove any empty quotes or extra spaces
    parts = [part.strip().strip("'\"") for part in text.split(',') if part.strip().strip("'\"")]

    # Step 3: Join the non-empty parts with a space
    text = ' '.join(parts)

    # Step 4: Ignore everything before the first colon
    if ':' in text:
        text = text.split(':', 1)[1]  # Keep content after the first colon

    # Step 5: Drop leading newline characters
    text = text.lstrip('\n')  # Removes any leading newlines

    # Step 6: Remove any leading numbers and spaces (e.g., '\n\n1. ')
    text = re.sub(r'^\s*\\n\\n\d+\.\s*', '', text)

    # Step 7: Replace any remaining newline characters with a space and strip leading/trailing spaces
    text = text.replace('\n', ' ').strip()

    # Step 8: If the text ends with a full stop, keep content only until the full stop
    if text.endswith('.'):
        text = text.split('.')[0] + '.'  # Keep content up to and including the first full stop

    return text

def preprocess_dataframe(df, column_name):
    # Apply the preprocess_text function to the specified column
    df[column_name] = df[column_name].apply(preprocess_text)
    return df

# Example usage
processed_df = preprocess_dataframe(merged_df, 'Meltemi_Instruct_4shot')

In [ ]:
processed_df['Meltemi_Instruct_4shot'].replace("', '', '', '', '', '', '', '', '',", np.nan, inplace=True)
processed_df['Meltemi_Instruct_4shot'].replace('', np.nan, inplace=True)

# Drop rows where 'Meltemi instruct' is NaN
processed_df = processed_df.dropna(subset=['Meltemi_Instruct_4shot'])
len(processed_df)

# Text preprocessing for Rouge computation (stemming:python implementation of Greek Stemmer from Ntais, 2006 )

In [ ]:
__cases = dict()
__cases["ΦΑΓΙΑ"] = "ΦΑ"
__cases["ΦΑΓΙΟΥ"] = "ΦΑ"
__cases["ΦΑΓΙΩΝ"] = "ΦΑ"
__cases["ΣΚΑΓΙΑ"] = "ΣΚΑ"
__cases["ΣΚΑΓΙΟΥ"] = "ΣΚΑ"
__cases["ΣΚΑΓΙΩΝ"] = "ΣΚΑ"
__cases["ΟΛΟΓΙΟΥ"] = "ΟΛΟ"
__cases["ΟΛΟΓΙΑ"] = "ΟΛΟ"
__cases["ΟΛΟΓΙΩΝ"] = "ΟΛΟ"
__cases["ΣΟΓΙΟΥ"] = "ΣΟ"
__cases["ΣΟΓΙΑ"] = "ΣΟ"
__cases["ΣΟΓΙΩΝ"] = "ΣΟ"
__cases["ΤΑΤΟΓΙΑ"] = "ΤΑΤΟ"
__cases["ΤΑΤΟΓΙΟΥ"] = "ΤΑΤΟ"
__cases["ΤΑΤΟΓΙΩΝ"] = "ΤΑΤΟ"
__cases["ΚΡΕΑΣ"] = "ΚΡΕ"
__cases["ΚΡΕΑΤΟΣ"] = "ΚΡΕ"
__cases["ΚΡΕΑΤΑ"] = "ΚΡΕ"
__cases["ΚΡΕΑΤΩΝ"] = "ΚΡΕ"
__cases["ΠΕΡΑΣ"] = "ΠΕΡ"
__cases["ΠΕΡΑΤΟΣ"] = "ΠΕΡ"
__cases["ΠΕΡΑΤΑ"] = "ΠΕΡ"
__cases["ΠΕΡΑΤΩΝ"] = "ΠΕΡ"
__cases["ΤΕΡΑΣ"] = "ΤΕΡ"
__cases["ΤΕΡΑΤΟΣ"] = "ΤΕΡ"
__cases["ΤΕΡΑΤΑ"] = "ΤΕΡ"
__cases["ΤΕΡΑΤΩΝ"] = "ΤΕΡ"
__cases["ΦΩΣ"] = "ΦΩ"
__cases["ΦΩΤΟΣ"] = "ΦΩ"
__cases["ΦΩΤΑ"] = "ΦΩ"
__cases["ΦΩΤΩΝ"] = "ΦΩ"
__cases["ΚΑΘΕΣΤΩΣ"] = "ΚΑΘΕΣΤ"
__cases["ΚΑΘΕΣΤΩΤΟΣ"] = "ΚΑΘΕΣΤ"
__cases["ΚΑΘΕΣΤΩΤΑ"] = "ΚΑΘΕΣΤ"
__cases["ΚΑΘΕΣΤΩΤΩΝ"] = "ΚΑΘΕΣΤ"
__cases["ΓΕΓΟΝΟΣ"] = "ΓΕΓΟΝ"
__cases["ΓΕΓΟΝΟΤΟΣ"] = "ΓΕΓΟΝ"
__cases["ΓΕΓΟΝΟΤΑ"] = "ΓΕΓΟΝ"
__cases["ΓΕΓΟΝΟΤΩΝ"] = "ΓΕΓΟΝ"
__vowels = "[ΑΕΗΙΟΥΩ]"
__refinedVowels = "[ΑΕΗΙΟΩ]"


def stemWord(w: str, exceptions: dict = None):
    stem = None
    suffix = None
    test1 = True

    if exceptions is not None and w in exceptions.keys():
        return exceptions[w]

    if len(w) < 4:
        return w

    pattern = None
    pattern2 = None
    pattern3 = None
    pattern4 = None

    # Step1
    pattern = re.compile(
        r"(.*)(ΦΑΓΙΑ|ΦΑΓΙΟΥ|ΦΑΓΙΩΝ|ΣΚΑΓΙΑ|ΣΚΑΓΙΟΥ|ΣΚΑΓΙΩΝ|ΟΛΟΓΙΟΥ|ΟΛΟΓΙΑ|ΟΛΟΓΙΩΝ|ΣΟΓΙΟΥ|ΣΟΓΙΑ|ΣΟΓΙΩΝ|ΤΑΤΟΓΙΑ|ΤΑΤΟΓΙΟΥ|ΤΑΤΟΓΙΩΝ|ΚΡΕΑΣ|ΚΡΕΑΤΟΣ|ΚΡΕΑΤΑ|ΚΡΕΑΤΩΝ|ΠΕΡΑΣ|ΠΕΡΑΤΟΣ|ΠΕΡΑΤΑ|ΠΕΡΑΤΩΝ|ΤΕΡΑΣ|ΤΕΡΑΤΟΣ|ΤΕΡΑΤΑ|ΤΕΡΑΤΩΝ|ΦΩΣ|ΦΩΤΟΣ|ΦΩΤΑ|ΦΩΤΩΝ|ΚΑΘΕΣΤΩΣ|ΚΑΘΕΣΤΩΤΟΣ|ΚΑΘΕΣΤΩΤΑ|ΚΑΘΕΣΤΩΤΩΝ|ΓΕΓΟΝΟΣ|ΓΕΓΟΝΟΤΟΣ|ΓΕΓΟΝΟΤΑ|ΓΕΓΟΝΟΤΩΝ)$")

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        suffix = fp[1]
        w = stem + __cases[suffix]
        test1 = False

    # Step 2a
    pattern = re.compile(r"^(.+?)(ΑΔΕΣ|ΑΔΩΝ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        reg1 = re.compile(r"(ΟΚ|ΜΑΜ|ΜΑΝ|ΜΠΑΜΠ|ΠΑΤΕΡ|ΓΙΑΓΙ|ΝΤΑΝΤ|ΚΥΡ|ΘΕΙ|ΠΕΘΕΡ)$")

        if not reg1.match(w):
            w = w + "ΑΔ"

    # Step 2b
    pattern2 = re.compile(r"^(.+?)(ΕΔΕΣ|ΕΔΩΝ)$")
    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        except2 = re.compile(r"(ΟΠ|ΙΠ|ΕΜΠ|ΥΠ|ΓΗΠ|ΔΑΠ|ΚΡΑΣΠ|ΜΙΛ)$")
        if except2.match(w):
            w = w + "ΕΔ"

    # Step 2c
    pattern3 = re.compile(r"^(.+?)(ΟΥΔΕΣ|ΟΥΔΩΝ)$")
    if pattern3.match(w):
        fp = pattern3.match(w).groups()
        stem = fp[0]
        w = stem
        except3 = re.compile(r"(ΑΡΚ|ΚΑΛΙΑΚ|ΠΕΤΑΛ|ΛΙΧ|ΠΛΕΞ|ΣΚ|Σ|ΦΛ|ΦΡ|ΒΕΛ|ΛΟΥΛ|ΧΝ|ΣΠ|ΤΡΑΓ|ΦΕ)$")
        if except3.match(w):
            w = w + "ΟΥΔ"

    # Step 2d
    pattern4 = re.compile("^(.+?)(ΕΩΣ|ΕΩΝ)$")
    if pattern4.match(w):
        fp = pattern4.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except4 = re.compile(r"^(Θ|Δ|ΕΛ|ΓΑΛ|Ν|Π|ΙΔ|ΠΑΡ)$")
        if except4.match(w):
            w = w + "Ε"

    # Step 3
    pattern = re.compile(r"^(.+?)(ΙΑ|ΙΟΥ|ΙΩΝ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        pattern2 = re.compile(__vowels + "$")
        test1 = False
        if pattern2.match(w):
            w = stem + "Ι"

    # Step 4
    pattern = re.compile(r"^(.+?)(ΙΚΑ|ΙΚΟ|ΙΚΟΥ|ΙΚΩΝ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern2 = re.compile(__vowels + "$")
        except5 = re.compile(
            r"^(ΑΛ|ΑΔ|ΕΝΔ|ΑΜΑΝ|ΑΜΜΟΧΑΛ|ΗΘ|ΑΝΗΘ|ΑΝΤΙΔ|ΦΥΣ|ΒΡΩΜ|ΓΕΡ|ΕΞΩΔ|ΚΑΛΠ|ΚΑΛΛΙΝ|ΚΑΤΑΔ|ΜΟΥΛ|ΜΠΑΝ|ΜΠΑΓΙΑΤ|ΜΠΟΛ|ΜΠΟΣ|ΝΙΤ|ΞΙΚ|ΣΥΝΟΜΗΛ|ΠΕΤΣ|ΠΙΤΣ|ΠΙΚΑΝΤ|ΠΛΙΑΤΣ|ΠΟΣΤΕΛΝ|ΠΡΩΤΟΔ|ΣΕΡΤ|ΣΥΝΑΔ|ΤΣΑΜ|ΥΠΟΔ|ΦΙΛΟΝ|ΦΥΛΟΔ|ΧΑΣ)$")
        if except5.match(w) or pattern2.match(w):
            w = w + "ΙΚ"

    # step 5a
    pattern = re.compile(r"^(.+?)(ΑΜΕ)$")
    pattern2 = re.compile(r"^(.+?)(ΑΓΑΜΕ|ΗΣΑΜΕ|ΟΥΣΑΜΕ|ΗΚΑΜΕ|ΗΘΗΚΑΜΕ)$")
    if w == "ΑΓΑΜΕ":
        w = "ΑΓΑΜ"

    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except6 = re.compile(r"^(ΑΝΑΠ|ΑΠΟΘ|ΑΠΟΚ|ΑΠΟΣΤ|ΒΟΥΒ|ΞΕΘ|ΟΥΛ|ΠΕΘ|ΠΙΚΡ|ΠΟΤ|ΣΙΧ|Χ)$")
        if except6.match(w):
            w = w + "ΑΜ"

    # Step 5b
    pattern2 = re.compile(r"^(.+?)(ΑΝΕ)$")
    pattern3 = re.compile(r"^(.+?)(ΑΓΑΝΕ|ΗΣΑΝΕ|ΟΥΣΑΝΕ|ΙΟΝΤΑΝΕ|ΙΟΤΑΝΕ|ΙΟΥΝΤΑΝΕ|ΟΝΤΑΝΕ|ΟΤΑΝΕ|ΟΥΝΤΑΝΕ|ΗΚΑΝΕ|ΗΘΗΚΑΝΕ)$")
    if pattern3.match(w):
        fp = pattern3.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern3 = re.compile(r"^(ΤΡ|ΤΣ)$")
        if pattern3.match(w):
            w = w + "ΑΓΑΝ"

    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern2 = re.compile(__refinedVowels + "$")
        except7 = re.compile(
            r"^(ΒΕΤΕΡ|ΒΟΥΛΚ|ΒΡΑΧΜ|Γ|ΔΡΑΔΟΥΜ|Θ|ΚΑΛΠΟΥΖ|ΚΑΣΤΕΛ|ΚΟΡΜΟΡ|ΛΑΟΠΛ|ΜΩΑΜΕΘ|Μ|ΜΟΥΣΟΥΛΜ|Ν|ΟΥΛ|Π|ΠΕΛΕΚ|ΠΛ|ΠΟΛΙΣ|ΠΟΡΤΟΛ|ΣΑΡΑΚΑΤΣ|ΣΟΥΛΤ|ΤΣΑΡΛΑΤ|ΟΡΦ|ΤΣΙΓΓ|ΤΣΟΠ|ΦΩΤΟΣΤΕΦ|Χ|ΨΥΧΟΠΛ|ΑΓ|ΟΡΦ|ΓΑΛ|ΓΕΡ|ΔΕΚ|ΔΙΠΛ|ΑΜΕΡΙΚΑΝ|ΟΥΡ|ΠΙΘ|ΠΟΥΡΙΤ|Σ|ΖΩΝΤ|ΙΚ|ΚΑΣΤ|ΚΟΠ|ΛΙΧ|ΛΟΥΘΗΡ|ΜΑΙΝΤ|ΜΕΛ|ΣΙΓ|ΣΠ|ΣΤΕΓ|ΤΡΑΓ|ΤΣΑΓ|Φ|ΕΡ|ΑΔΑΠ|ΑΘΙΓΓ|ΑΜΗΧ|ΑΝΙΚ|ΑΝΟΡΓ|ΑΠΗΓ|ΑΠΙΘ|ΑΤΣΙΓΓ|ΒΑΣ|ΒΑΣΚ|ΒΑΘΥΓΑΛ|ΒΙΟΜΗΧ|ΒΡΑΧΥΚ|ΔΙΑΤ|ΔΙΑΦ|ΕΝΟΡΓ|ΘΥΣ|ΚΑΠΝΟΒΙΟΜΗΧ|ΚΑΤΑΓΑΛ|ΚΛΙΒ|ΚΟΙΛΑΡΦ|ΛΙΒ|ΜΕΓΛΟΒΙΟΜΗΧ|ΜΙΚΡΟΒΙΟΜΗΧ|ΝΤΑΒ|ΞΗΡΟΚΛΙΒ|ΟΛΙΓΟΔΑΜ|ΟΛΟΓΑΛ|ΠΕΝΤΑΡΦ|ΠΕΡΗΦ|ΠΕΡΙΤΡ|ΠΛΑΤ|ΠΟΛΥΔΑΠ|ΠΟΛΥΜΗΧ|ΣΤΕΦ|ΤΑΒ|ΤΕΤ|ΥΠΕΡΗΦ|ΥΠΟΚΟΠ|ΧΑΜΗΛΟΔΑΠ|ΨΗΛΟΤΑΒ)$")
        if (pattern2.match(w)) or (except7.match(w)):
            w = w + "ΑΝ"

    # //Step 5c
    pattern3 = re.compile(r"^(.+?)(ΕΤΕ)$")
    pattern4 = re.compile(r"^(.+?)(ΗΣΕΤΕ)$")
    if pattern4.match(w):
        fp = pattern4.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False

    if pattern3.match(w):
        fp = pattern3.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern3 = re.compile(__refinedVowels + "$")
        except8 = re.compile(
            r"(ΟΔ|ΑΙΡ|ΦΟΡ|ΤΑΘ|ΔΙΑΘ|ΣΧ|ΕΝΔ|ΕΥΡ|ΤΙΘ|ΥΠΕΡΘ|ΡΑΘ|ΕΝΘ|ΡΟΘ|ΣΘ|ΠΥΡ|ΑΙΝ|ΣΥΝΔ|ΣΥΝ|ΣΥΝΘ|ΧΩΡ|ΠΟΝ|ΒΡ|ΚΑΘ|ΕΥΘ|ΕΚΘ|ΝΕΤ|ΡΟΝ|ΑΡΚ|ΒΑΡ|ΒΟΛ|ΩΦΕΛ)$")
        except9 = re.compile(
            r"^(ΑΒΑΡ|ΒΕΝ|ΕΝΑΡ|ΑΒΡ|ΑΔ|ΑΘ|ΑΝ|ΑΠΛ|ΒΑΡΟΝ|ΝΤΡ|ΣΚ|ΚΟΠ|ΜΠΟΡ|ΝΙΦ|ΠΑΓ|ΠΑΡΑΚΑΛ|ΣΕΡΠ|ΣΚΕΛ|ΣΥΡΦ|ΤΟΚ|Υ|Δ|ΕΜ|ΘΑΡΡ|Θ)$")
        if (pattern3.match(w)) or (except8.match(w)) or (except9.match(w)):
            w = w + "ΕΤ"

    # Step 5d
    pattern = re.compile(r"^(.+?)(ΟΝΤΑΣ|ΩΝΤΑΣ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except10 = re.compile(r"^(ΑΡΧ)$")
        except11 = re.compile(r"(ΚΡΕ)$")
        if except10.match(w):
            w = w + "ΟΝΤ"
        if except11.match(w):
            w = w + "ΩΝΤ"

    # Step 5e
    pattern = re.compile(r"^(.+?)(ΟΜΑΣΤΕ|ΙΟΜΑΣΤΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except11 = re.compile("^(ΟΝ)$")
        if except11.match(w):
            w = w + "ΟΜΑΣΤ"

    # Step 5f
    pattern = re.compile(r"^(.+?)(ΕΣΤΕ)$")
    pattern2 = re.compile(r"^(.+?)(ΙΕΣΤΕ)$")
    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        pattern2 = re.compile(r"^(Π|ΑΠ|ΣΥΜΠ|ΑΣΥΜΠ|ΑΚΑΤΑΠ|ΑΜΕΤΑΜΦ)$")
        if pattern2.match(w):
            w = w + "ΙΕΣΤ"

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except12 = re.compile(r"^(ΑΛ|ΑΡ|ΕΚΤΕΛ|Ζ|Μ|Ξ|ΠΑΡΑΚΑΛ|ΑΡ|ΠΡΟ|ΝΙΣ)$")
        if except12.match(w):
            w = w + "ΕΣΤ"

    # Step 5g
    pattern = re.compile(r"^(.+?)(ΗΚΑ|ΗΚΕΣ|ΗΚΕ)$")
    pattern2 = re.compile(r"^(.+?)(ΗΘΗΚΑ|ΗΘΗΚΕΣ|ΗΘΗΚΕ)$")
    if pattern2.match(w):
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except13 = re.compile(r"(ΣΚΩΛ|ΣΚΟΥΛ|ΝΑΡΘ|ΣΦ|ΟΘ|ΠΙΘ)$")
        except14 = re.compile(r"^(ΔΙΑΘ|Θ|ΠΑΡΑΚΑΤΑΘ|ΠΡΟΣΘ|ΣΥΝΘ|)$")
        if (except13.match(w)) or (except14.match(w)):
            w = w + "ΗΚ"

    # Step 5h
    pattern = re.compile(r"^(.+?)(ΟΥΣΑ|ΟΥΣΕΣ|ΟΥΣΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except15 = re.compile(
            r"^(ΦΑΡΜΑΚ|ΧΑΔ|ΑΓΚ|ΑΝΑΡΡ|ΒΡΟΜ|ΕΚΛΙΠ|ΛΑΜΠΙΔ|ΛΕΧ|Μ|ΠΑΤ|Ρ|Λ|ΜΕΔ|ΜΕΣΑΖ|ΥΠΟΤΕΙΝ|ΑΜ|ΑΙΘ|ΑΝΗΚ|ΔΕΣΠΟΖ|ΕΝΔΙΑΦΕΡ|ΔΕ|ΔΕΥΤΕΡΕΥ|ΚΑΘΑΡΕΥ|ΠΛΕ|ΤΣΑ)$")
        except16 = re.compile(r"(ΠΟΔΑΡ|ΒΛΕΠ|ΠΑΝΤΑΧ|ΦΡΥΔ|ΜΑΝΤΙΛ|ΜΑΛΛ|ΚΥΜΑΤ|ΛΑΧ|ΛΗΓ|ΦΑΓ|ΟΜ|ΠΡΩΤ)$")
        if (except15.match(w)) or (except16.match(w)):
            w = w + "ΟΥΣ"

    # Step 5i
    pattern = re.compile(r"^(.+?)(ΑΓΑ|ΑΓΕΣ|ΑΓΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except17 = re.compile(r"^(ΨΟΦ|ΝΑΥΛΟΧ)$")
        except20 = re.compile(r"(ΚΟΛΛ)$")
        except18 = re.compile(
            r"^(ΑΒΑΣΤ|ΠΟΛΥΦ|ΑΔΗΦ|ΠΑΜΦ|Ρ|ΑΣΠ|ΑΦ|ΑΜΑΛ|ΑΜΑΛΛΙ|ΑΝΥΣΤ|ΑΠΕΡ|ΑΣΠΑΡ|ΑΧΑΡ|ΔΕΡΒΕΝ|ΔΡΟΣΟΠ|ΞΕΦ|ΝΕΟΠ|ΝΟΜΟΤ|ΟΛΟΠ|ΟΜΟΤ|ΠΡΟΣΤ|ΠΡΟΣΩΠΟΠ|ΣΥΜΠ|ΣΥΝΤ|Τ|ΥΠΟΤ|ΧΑΡ|ΑΕΙΠ|ΑΙΜΟΣΤ|ΑΝΥΠ|ΑΠΟΤ|ΑΡΤΙΠ|ΔΙΑΤ|ΕΝ|ΕΠΙΤ|ΚΡΟΚΑΛΟΠ|ΣΙΔΗΡΟΠ|Λ|ΝΑΥ|ΟΥΛΑΜ|ΟΥΡ|Π|ΤΡ|Μ)$")
        except19 = re.compile(r"(ΟΦ|ΠΕΛ|ΧΟΡΤ|ΛΛ|ΣΦ|ΡΠ|ΦΡ|ΠΡ|ΛΟΧ|ΣΜΗΝ)$")
        if (except18.match(w) and except19.match(w)) and not ((except17.match(w)) or (except20.match(w))):
            w = w + "ΑΓ"

    # Step 5j
    pattern = re.compile("^(.+?)(ΗΣΕ|ΗΣΟΥ|ΗΣΑ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except21 = re.compile(r"^(Ν|ΧΕΡΣΟΝ|ΔΩΔΕΚΑΝ|ΕΡΗΜΟΝ|ΜΕΓΑΛΟΝ|ΕΠΤΑΝ)$")
        if except21.match(w):
            w = w + "ΗΣ"

    # Step 5k
    pattern = re.compile(r"^(.+?)(ΗΣΤΕ)$")

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except22 = re.compile(r"^(ΑΣΒ|ΣΒ|ΑΧΡ|ΧΡ|ΑΠΛ|ΑΕΙΜΝ|ΔΥΣΧΡ|ΕΥΧΡ|ΚΟΙΝΟΧΡ|ΠΑΛΙΜΨ)$")
        if except22.match(w):
            w = w + "ΗΣΤ"

    # Step 5l
    pattern = re.compile("^(.+?)(ΟΥΝΕ|ΗΣΟΥΝΕ|ΗΘΟΥΝΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except23 = re.compile("^(Ν|Ρ|ΣΠΙ|ΣΤΡΑΒΟΜΟΥΤΣ|ΚΑΚΟΜΟΥΤΣ|ΕΞΩΝ)$")
        if except23.match(w):
            w = w + "ΟΥΝ"

    # Step 5l
    pattern = re.compile(r"^(.+?)(ΟΥΜΕ|ΗΣΟΥΜΕ|ΗΘΟΥΜΕ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem
        test1 = False
        except24 = re.compile(r"^(ΠΑΡΑΣΟΥΣ|Φ|Χ|ΩΡΙΟΠΛ|ΑΖ|ΑΛΛΟΣΟΥΣ|ΑΣΟΥΣ)$")
        if except24.match(w):
            w = w + "ΟΥΜ"

    # Step 6
    pattern = re.compile(r"^(.+?)(ΜΑΤΑ|ΜΑΤΩΝ|ΜΑΤΟΣ)$")
    pattern2 = re.compile(
        r"^(.+?)(Α|ΑΓΑΤΕ|ΑΓΑΝ|ΑΕΙ|ΑΜΑΙ|ΑΝ|ΑΣ|ΑΣΑΙ|ΑΤΑΙ|ΑΩ|Ε|ΕΙ|ΕΙΣ|ΕΙΤΕ|ΕΣΑΙ|ΕΣ|ΕΤΑΙ|Ι|ΙΕΜΑΙ|ΙΕΜΑΣΤΕ|ΙΕΤΑΙ|ΙΕΣΑΙ|ΙΕΣΑΣΤΕ|ΙΟΜΑΣΤΑΝ|ΙΟΜΟΥΝ|ΙΟΜΟΥΝΑ|ΙΟΝΤΑΝ|ΙΟΝΤΟΥΣΑΝ|ΙΟΣΑΣΤΑΝ|ΙΟΣΑΣΤΕ|ΙΟΣΟΥΝ|ΙΟΣΟΥΝΑ|ΙΟΤΑΝ|ΙΟΥΜΑ|ΙΟΥΜΑΣΤΕ|ΙΟΥΝΤΑΙ|ΙΟΥΝΤΑΝ|Η|ΗΔΕΣ|ΗΔΩΝ|ΗΘΕΙ|ΗΘΕΙΣ|ΗΘΕΙΤΕ|ΗΘΗΚΑΤΕ|ΗΘΗΚΑΝ|ΗΘΟΥΝ|ΗΘΩ|ΗΚΑΤΕ|ΗΚΑΝ|ΗΣ|ΗΣΑΝ|ΗΣΑΤΕ|ΗΣΕΙ|ΗΣΕΣ|ΗΣΟΥΝ|ΗΣΩ|Ο|ΟΙ|ΟΜΑΙ|ΟΜΑΣΤΑΝ|ΟΜΟΥΝ|ΟΜΟΥΝΑ|ΟΝΤΑΙ|ΟΝΤΑΝ|ΟΝΤΟΥΣΑΝ|ΟΣ|ΟΣΑΣΤΑΝ|ΟΣΑΣΤΕ|ΟΣΟΥΝ|ΟΣΟΥΝΑ|ΟΤΑΝ|ΟΥ|ΟΥΜΑΙ|ΟΥΜΑΣΤΕ|ΟΥΝ|ΟΥΝΤΑΙ|ΟΥΝΤΑΝ|ΟΥΣ|ΟΥΣΑΝ|ΟΥΣΑΤΕ|Υ|ΥΣ|Ω|ΩΝ)$")

    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem + "ΜΑ"

    if pattern2.match(w) and test1:
        fp = pattern2.match(w).groups()
        stem = fp[0]
        w = stem

    # Step 7 (ΠΑΡΑΘΕΤΙΚΑ)
    pattern = re.compile(r"^(.+?)(ΕΣΤΕΡ|ΕΣΤΑΤ|ΟΤΕΡ|ΟΤΑΤ|ΥΤΕΡ|ΥΤΑΤ|ΩΤΕΡ|ΩΤΑΤ)$")
    if pattern.match(w):
        fp = pattern.match(w).groups()
        stem = fp[0]
        w = stem

    return w

# Notes:
# (1) Greek word to get stemmed should be in capital letters and without accents. E.g. ΨΩΜΙ and not Ψωμί
# (2) Any word of length 1 - 3 will not get stemmed.
# (3) The exceptions argument is a dictionary where a key is a word and a value is a custom stem of your liking

In [ ]:
def replaceMultiple(main, replacements, new):
  for elem in replacements:
    if elem in main:
      main = main.replace(elem, new)
  return main

def normalize(x):
  x = replaceMultiple(x, ['ά', 'ὰ', 'ἀ','ἁ','ἂ','ἃ','ἄ','ἅ','ἆ','ἇ','ᾰ','ᾱ','ᾲ','ᾳ','ᾴ','ᾶ','ᾷ'], 'α')
  x = replaceMultiple(x, ['έ','ὲ','ἐ','ἑ','ἒ','ἓ','ἔ','ἕ'], 'ε')
  x = replaceMultiple(x, ['ή','ὴ','ἠ','ἡ','ἢ','ἣ','ἤ','ἥ','ἦ','ἧ','ῂ','ῃ','ῄ','ῆ','ῇ'], 'η')
  x = replaceMultiple(x, ['ί','ὶ','ἰ','ἱ','ἲ','ἳ','ἴ','ἵ','ἶ','ἷ','ῐ','ῑ','ῒ','ΐ','ῖ','ῗ'], 'ι')
  x = replaceMultiple(x, ['ὸ','ό','ὀ','ὁ','ὂ','ὃ','ὄ','ὅ'], 'ο')
  x = replaceMultiple(x, ['ύ','ὺ','ΰ', 'ϋ','ὐ','ὑ','ὒ','ὓ','ὔ','ὕ','ὖ','ὗ','ῠ','ῡ','ῢ','ΰ','ῦ'], 'υ')
  x = replaceMultiple(x, ['ώ','ὼ','ῶ','ῲ','ῳ','ῴ','ῶ','ῷ','ὠ','ὡ','ὢ','ὣ','ὤ','ὥ','ὦ','ὧ'], 'ω')
  return x

def remove_emojis(text):
    return demoji.replace(text, '')

def sep_punc(x):
    punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~؛،؟؛.»«”'
    out = []
    for char in x:
        if char in punc:
            out.append(' ' + char + ' ')
        else:
            out.append(char)
    return ''.join(out)

def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return set(stopwords)

def preprocess_text(text): #,stopwords_file):
      # Load stopwords
    #stopwords = load_stopwords(stopwords_file)
    # Remove retweets
    text = re.sub(r'^RT[\s]+', '', text, flags=re.IGNORECASE)
    # Remove usernames @
    text = re.sub(r'@[^\s]+', '', text)
    # Remove URLs
    text = re.sub(r'https\S+', '', text)
    url_words = ['url', 'URL', 'html', 'HTML', 'http', 'HTTP']
    for u in url_words:
        text = re.sub(u, '', text)
    text = text.lower()
    text=normalize(text)
     # Separate punctuation characters
    text = sep_punc(text)
    # Remove punctuation characters
    text = re.sub(f"[{''.join(re.escape(c) for c in string.punctuation if c != '_') }]", ' ', text)
    # Remove Latin characters
    text = re.sub(r'[a-zA-Z]+', '', text)
    # Remove numbers
    text = re.sub(r'\b\d+\b', '', text)
  # Remove words with character length equal to or less than 1
    words = text.split()
    filtered_words = [word for word in words if len(word) > 1]
    text = ' '.join(filtered_words)
    text = remove_emojis(text)
    text = str(text).upper().strip()
    #words = text.split()
    #filtered_words = [word for word in words if word not in stopwords]
    #text = ' '.join(filtered_words)
    tokenized_tweet = word_tokenize(text)
    tokenized_tweet = [stemWord(i) for i in tokenized_tweet]
    return ' '.join(tokenized_tweet)

# reference keypoints

In [ ]:
ref_df = pd.read_csv('/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/train_dev_test_dataset/key_points_human_translated_test.csv')

In [ ]:
pd.set_option("display.max_colwidth", None)

def preprocess_dataset(pred_df, ref_df):
    # Dynamically extract unique topics and stances from the pred_df
    topics = pred_df['topic'].unique()
    stances = pred_df['stance'].unique()

    predictions, references = [], []

    for topic in topics:
        for stance in stances:
            # Filter predicted keypoints from pred_df based on dynamic topic and stance
            kps = pred_df.loc[(pred_df['topic'] == topic) & (pred_df['stance'] == stance), 'Meltemi_Instruct_4shot'].tolist()

            # Filter reference keypoints from ref_df based on dynamic topic and stance
            gold_kps = ref_df.loc[(ref_df['topic'] == topic) & (ref_df['stance'] == stance), 'key_point'].tolist()

            predictions.append(kps)
            references.append(gold_kps)

    return predictions, references

In [ ]:
predictions, references = preprocess_dataset(processed_df, ref_df)
print(predictions[0])
print(references[0])

In [ ]:
def compute_rouge(predictions, references):   #,stopwords_file):
    # Load the Rouge metric from the evaluate library
    rouge = load('rouge')

    rouge1_scores, rouge2_scores, rougel_scores = [], [], []

    for preds, refs in zip(predictions, references):
        # Compute per topic average scores on all unique combinations of generated keypoints and ground-truth
        r_1, r_2, r_l = [], [], []
        # Preprocess the predicted keypoints and reference keypoints
        preds = [preprocess_text(pred) for pred in preds]
        print(preds)
        refs = [preprocess_text(ref) for ref in refs]
        print(refs)
        for a, b in product(preds, refs):
            # Use the evaluate library's rouge.compute() function to get the scores for each keypoint pair
            rouge_scores = rouge.compute(predictions=[a], references=[b], tokenizer=lambda x: x.split())

            # Since the values are floats, no need to access 'fmeasure'
            r_1.append(round(rouge_scores['rouge1'], 3))
            r_2.append(round(rouge_scores['rouge2'], 3))
            r_l.append(round(rouge_scores['rougeL'], 3))

        # Save per topic scores to compute average over all topics as the final score
        rouge1_scores.append(round(mean(r_1), 3))
        rouge2_scores.append(round(mean(r_2), 3))
        rougel_scores.append(round(mean(r_l), 3))

    # Print the average scores across all topics
    print("Rouge 1: {}".format(round(mean(rouge1_scores), 3)))
    print("Rouge 2: {}".format(round(mean(rouge2_scores), 3)))
    print("Rouge L: {}".format(round(mean(rougel_scores), 3)))

def compute_bertscore(predictions, references):
    bertscore_metric = load('bertscore')

    bertscore_precisions, bertscore_recalls, bertscore_f1s = [], [], []

    for preds, refs in zip(predictions, references):
        # Compute per-topic average scores on all unique combinations of generated keypoints and ground-truth
        r_1, r_2, r_l = [], [], []
        bert_prec, bert_recall, bert_f1 = [], [], []


        # Compute Rouge Scores
        for a, b in product(preds, refs):
            # Compute BERTScore without preprocessing
            bertscore_result = bertscore_metric.compute(predictions=[a], references=[b], lang="el")
            bert_prec.append(round(bertscore_result['precision'][0], 3))
            bert_recall.append(round(bertscore_result['recall'][0], 3))
            bert_f1.append(round(bertscore_result['f1'][0], 3))


        # Save per-topic BERTScore to compute average over all topics as the final score
        bertscore_precisions.append(round(mean(bert_prec), 3))
        bertscore_recalls.append(round(mean(bert_recall), 3))
        bertscore_f1s.append(round(mean(bert_f1), 3))

    # Print the average BERTScore scores across all topics
    print("BERTScore Precision: {}".format(round(mean(bertscore_precisions), 3)))
    print("BERTScore Recall: {}".format(round(mean(bertscore_recalls), 3)))
    print("BERTScore F1: {}".format(round(mean(bertscore_f1s), 3)))



In [ ]:
compute_rouge(predictions, references)  

In [ ]:
compute_bertscore(predictions, references)